In [1]:
from openai import AzureOpenAI
import os
from dotenv import load_dotenv
load_dotenv()

True

Azure-Hack-2024/.env内にAZURE_OPENAI_ENDPOINTとAZURE_OPENAI_API_KEYを置いている前提

In [2]:
# Azure OpenAIのGPT-4を使って文章生成できるかテスト
client = AzureOpenAI(
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"), 
  api_key=os.getenv("AZURE_OPENAI_API_KEY"),  
  api_version="2024-02-01"
)

In [3]:

response = client.chat.completions.create(
    model="gpt-4", # model = "deployment_name".
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Does Azure OpenAI support customer managed keys?"},
        {"role": "assistant", "content": "Yes, customer managed keys are supported by Azure OpenAI."},
        {"role": "user", "content": "Do other Azure AI services support this too?"}
    ]
)

print(response.choices[0].message.content)

Yes, several other Azure AI services support customer-managed keys (CMK) for enhanced security and compliance. Customer-managed keys allow users to control and manage the encryption keys used to encrypt data at rest in the cloud, providing an additional layer of security and compliance.

For example:

1. **Azure Cognitive Services**: Some services within Azure Cognitive Services allow customers to encrypt data at rest using their own keys managed in Azure Key Vault. This feature provides additional control over the security of the data processed by these services.
   
2. **Azure Machine Learning**: Supports customer-managed keys for encrypting data in the workspace, providing additional control and security over the machine learning assets and data stored in Azure.

The availability and implementation of customer-managed keys can vary between different Azure AI services and features within those services. It’s always a good practice to review the specific documentation for each Azure A

In [4]:
# azure ai search を使って検索できるかテスト
# https://pypi.org/project/azure-search-documents/
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient

In [5]:
service_endpoint = os.environ["AZURE_SEARCH_SERVICE_ENDPOINT"]
index_name = os.environ["AZURE_SEARCH_INDEX_NAME"]
key = os.environ["AZURE_SEARCH_API_KEY"]

In [6]:
search_client = SearchClient(service_endpoint, index_name, AzureKeyCredential(key))

In [7]:
results = search_client.search(search_text="nameがそうなんだ", top=5,logging_enable=True)

In [8]:
for result in results:
    print(result)

{'form': 'アローラのすがた', 'isMegaEvolution': False, 'no': 50, 'rid': 'UlpkN0FKSkcwcXRCQUFBQUFBQUFBQT090', 'id': 'a8051759-185a-4c2f-89f0-3a59d21e9a66', 'stats': {'hp': 10, 'attack': 55, 'defence': 30, 'spAttack': 35, 'spDefence': 45, 'speed': 90}, 'name': 'ディグダ', 'types': ['じめん', 'はがね'], 'hiddenAbilities': ['すなのちから'], 'evolutions': [51], 'abilities': ['すながくれ', 'カーリーヘアー'], '@search.score': 11.906606, '@search.reranker_score': None, '@search.highlights': None, '@search.captions': None}
{'form': '', 'isMegaEvolution': False, 'no': 232, 'rid': 'UlpkN0FKSkcwcXNQQVFBQUFBQUFBQT090', 'id': 'f0f8b825-091c-4067-9166-a20dafdfd28f', 'stats': {'hp': 90, 'attack': 120, 'defence': 120, 'spAttack': 60, 'spDefence': 60, 'speed': 50}, 'name': 'ドンファン', 'types': ['じめん'], 'hiddenAbilities': ['すながくれ'], 'evolutions': [], 'abilities': ['がんじょう'], '@search.score': 11.639136, '@search.reranker_score': None, '@search.highlights': None, '@search.captions': None}
{'form': '', 'isMegaEvolution': False, 'no': 75, 'rid': '